In [7]:
from PIL import Image
from numpy import asarray
import numpy as np
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from keras.utils import np_utils
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
print('all imported')

all imported


In [8]:
def readImage(filePath):
    img = Image.open(filePath)
    
    size=(30,40)
    #resize image
    out = img.resize(size)

    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(out)
    numpydata = np.repeat(numpydata[:, :, np.newaxis], 3, axis=2)
    # <class 'numpy.ndarray'>
    #print(type(numpydata))

    #  shape
    #print(numpydata.shape)
    return numpydata
print(readImage('data/nervus/ISIC_0001769.jpg'))

[[[[142 112  87]
   [142 112  87]
   [142 112  87]]

  [[157 121  91]
   [157 121  91]
   [157 121  91]]

  [[168 130  98]
   [168 130  98]
   [168 130  98]]

  ...

  [[174 139 113]
   [174 139 113]
   [174 139 113]]

  [[169 135 110]
   [169 135 110]
   [169 135 110]]

  [[159 130 108]
   [159 130 108]
   [159 130 108]]]


 [[[148 115  89]
   [148 115  89]
   [148 115  89]]

  [[161 123  95]
   [161 123  95]
   [161 123  95]]

  [[170 133 103]
   [170 133 103]
   [170 133 103]]

  ...

  [[175 139 114]
   [175 139 114]
   [175 139 114]]

  [[171 136 112]
   [171 136 112]
   [171 136 112]]

  [[164 132 111]
   [164 132 111]
   [164 132 111]]]


 [[[155 120  95]
   [155 120  95]
   [155 120  95]]

  [[165 129 102]
   [165 129 102]
   [165 129 102]]

  [[170 133 103]
   [170 133 103]
   [170 133 103]]

  ...

  [[178 144 120]
   [178 144 120]
   [178 144 120]]

  [[173 140 116]
   [173 140 116]
   [173 140 116]]

  [[168 136 115]
   [168 136 115]
   [168 136 115]]]


 ...


 [[[154 129 

In [9]:
listFiles = os.listdir('data/melanoma/')
print(listFiles[:5])

['ISIC_0012099.jpg', 'ISIC_0012151.jpg', 'ISIC_0012288.jpg', 'ISIC_0012434.jpg', 'ISIC_0013232.jpg']


In [10]:
#load Benign
listCancer = []
listResults = []
for elem in listFiles:
    img = readImage('data/melanoma/'+elem)
    listCancer += [img]
    listResults += [1]
print(listCancer[:3])
print(img.shape)

[array([[[[ 54,  55,  58],
         [ 54,  55,  58],
         [ 54,  55,  58]],

        [[168, 168, 169],
         [168, 168, 169],
         [168, 168, 169]],

        [[184, 184, 185],
         [184, 184, 185],
         [184, 184, 185]],

        ...,

        [[190, 180, 183],
         [190, 180, 183],
         [190, 180, 183]],

        [[172, 164, 169],
         [172, 164, 169],
         [172, 164, 169]],

        [[ 66,  72,  89],
         [ 66,  72,  89],
         [ 66,  72,  89]]],


       [[[ 99,  99, 102],
         [ 99,  99, 102],
         [ 99,  99, 102]],

        [[179, 178, 179],
         [179, 178, 179],
         [179, 178, 179]],

        [[185, 183, 184],
         [185, 183, 184],
         [185, 183, 184]],

        ...,

        [[192, 180, 184],
         [192, 180, 184],
         [192, 180, 184]],

        [[184, 172, 175],
         [184, 172, 175],
         [184, 172, 175]],

        [[100,  97, 108],
         [100,  97, 108],
         [100,  97, 108]]],


       

In [11]:
listFiles = os.listdir('data/nervus/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/nervus/'+elem)
    listResults += [0]
    listCancer += [img]

['ISIC_0001769.jpg', 'ISIC_0001852.jpg', 'ISIC_0001871.jpg', 'ISIC_0003462.jpg', 'ISIC_0003539.jpg']


In [12]:
listFiles = os.listdir('data/seborrheic_keratosis/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/seborrheic_keratosis/'+elem)
    listResults += [2]
    listCancer += [img]

['ISIC_0012143.jpg', 'ISIC_0012204.jpg', 'ISIC_0012210.jpg', 'ISIC_0012254.jpg', 'ISIC_0012380.jpg']


In [13]:
print(listResults)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [14]:
import numpy as np
lst = np.random.permutation(len(listResults))
lst = list(lst)

In [15]:
lst

[115,
 37,
 20,
 21,
 30,
 7,
 89,
 0,
 49,
 45,
 97,
 69,
 126,
 23,
 60,
 67,
 140,
 50,
 80,
 123,
 137,
 35,
 36,
 84,
 78,
 42,
 58,
 108,
 118,
 18,
 81,
 87,
 86,
 109,
 100,
 93,
 101,
 8,
 104,
 3,
 95,
 15,
 76,
 12,
 111,
 53,
 85,
 96,
 112,
 128,
 43,
 25,
 106,
 24,
 82,
 68,
 10,
 65,
 40,
 110,
 39,
 32,
 52,
 17,
 47,
 63,
 136,
 71,
 44,
 130,
 121,
 88,
 56,
 147,
 4,
 22,
 73,
 54,
 62,
 134,
 98,
 5,
 46,
 29,
 107,
 133,
 103,
 48,
 142,
 64,
 102,
 90,
 77,
 27,
 75,
 61,
 148,
 132,
 57,
 41,
 1,
 146,
 9,
 129,
 66,
 105,
 120,
 116,
 143,
 83,
 70,
 19,
 6,
 144,
 11,
 119,
 141,
 91,
 13,
 74,
 135,
 16,
 138,
 31,
 34,
 127,
 38,
 2,
 51,
 149,
 113,
 72,
 59,
 99,
 114,
 94,
 124,
 28,
 125,
 55,
 139,
 79,
 131,
 14,
 122,
 33,
 145,
 26,
 92,
 117]

In [16]:
data = [listResults[i] for i in lst]

In [17]:
sz = (len(lst) * 80) // 100
train = [data[i] for i in range(sz)]
test = [data[i] for i in range(sz, len(lst))]
print(train)
print(test)

[2, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 0, 0, 0, 2, 2, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 1, 0, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 0, 0, 1, 2, 1, 2, 0, 0, 2, 2, 2, 0, 0, 1, 1, 2, 1, 2, 2, 0, 1, 0]
[2, 1, 2, 0, 0, 2, 0, 1, 0, 2, 2, 0, 0, 0, 2, 0, 2, 1, 2, 0, 2, 0, 2, 1, 2, 0, 2, 1, 0, 2]


In [18]:
from statistics import mode
rsp = mode(train)
pred = []
for i in range(len(test)):
    pred+=[rsp]
acc = 0
for i in range(len(test)):
    if pred[i] == test[i]:
        acc+=1
acc = acc / len(test)
acc
precision = precision_score(test, pred, average='weighted')
recall = recall_score(test, pred, average='weighted')
print("acc: ", acc)
print("precision: ", precision)
print("recall: ", recall)

acc:  0.4
precision:  0.16000000000000003
recall:  0.4


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
